In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
odf = pd.read_csv('../data/original_convfinqa_dataset.csv')

In [3]:
odf.head()

,html_content,questions
0,"<html>\n <body>\n ""distribution date"" ) . unt...",['what was the percentage of the total common ...
1,"<html>\n <body>\n ""three factor formula"" ) . ...","['in the year of 2013, during the second quart..."
2,"<html>\n <body>\n $ 25.7 million in cash , in...","['in the year of 2009, how much did the income..."
3,<html>\n <body>\n $ 43.3 million in 2011 comp...,['what was the difference between the net sale...
4,<html>\n <body>\n ( $ 125 million ) and highe...,['what were north american industrial packagin...


In [4]:
odf = odf.drop('questions', axis='columns')

In [5]:
odf['index'] = odf.index + 1 

In [7]:
total_df = pd.read_csv('../data/json_encoded_convfinqa.csv')

In [8]:
merged_df = total_df.merge(odf, on=['index']).sort_index(axis=1)

In [9]:
merged_df.shape

(17612, 5)

In [10]:
merged_df.head()

,answer,html_content,index,json_encoding,question
0,17739,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under stock and option pl...
1,18851,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is total ?
2,842,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under the vertex purchase...
3,270,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under the vertex 401 ( k ...
4,$0.88,"<html>\n <body>\n ""three factor formula"" ) . ...",2,"<html>\n<body>\n ""three factor formula"" ) . t...",What is basic earnings ( loss ) per share for ...


In [11]:
merged_df.loc[0, 'question']

'What is common stock under stock and option plans ?'

In [12]:
merged_df.loc[0, 'answer']

'17739'

In [13]:
print(merged_df.loc[0, 'html_content'])

<html>
 <body>
  "distribution date" ) . until the distribution date ( or earlier redemption or expiration of the rights ) , the rights will be traded with , and only with , the common stock . until a right is exercised , the right will not entitle the holder thereof to any rights as a stockholder . if any person or group becomes an acquiring person , each holder of a right , other than rights beneficially owned by the acquiring person , will thereafter have the right to receive upon exercise and payment of the purchase price that number of shares of common stock having a market value of two times the purchase price and , if the company is acquired in a business combination transaction or 50% ( 50 % ) or more of its assets are sold , each holder of a right will thereafter have the right to receive upon exercise and payment of the purchase price that number of shares of common stock of the acquiring company which at the time of the transaction will have a market value of two times the p

In [14]:
print(merged_df.loc[0, 'json_encoding'])

<html>
<body>
  "distribution date" ) . until the distribution date ( or earlier redemption or expiration of the rights ) , the rights will be traded with , and only with , the common stock . until a right is exercised , the right will not entitle the holder thereof to any rights as a stockholder . if any person or group becomes an acquiring person , each holder of a right , other than rights beneficially owned by the acquiring person , will thereafter have the right to receive upon exercise and payment of the purchase price that number of shares of common stock having a market value of two times the purchase price and , if the company is acquired in a business combination transaction or 50% ( 50 % ) or more of its assets are sold , each holder of a right will thereafter have the right to receive upon exercise and payment of the purchase price that number of shares of common stock of the acquiring company which at the time of the transaction will have a market value of two times the pu

In [15]:
# merged_df = merged_df.drop('json_encoding', axis='columns')

In [16]:
merged_df.head()

,answer,html_content,index,json_encoding,question
0,17739,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under stock and option pl...
1,18851,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is total ?
2,842,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under the vertex purchase...
3,270,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under the vertex 401 ( k ...
4,$0.88,"<html>\n <body>\n ""three factor formula"" ) . ...",2,"<html>\n<body>\n ""three factor formula"" ) . t...",What is basic earnings ( loss ) per share for ...


In [17]:
def beautify_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table')
    return table

In [18]:
index_info_df = pd.read_csv('../data/index_info.csv')
index_info = index_info_df['Index of data without headers'].to_list()
len(index_info)

150

In [19]:
def convert_table_to_pseudo_json(row):
    # Parse HTML content
    soup = BeautifulSoup(row['html_content'], 'html.parser')

    # Find the table
    table = soup.find('table')

    if int(row['index']) in index_info:
        lst = []
        for rows in table.find_all('tr'):
            row = rows.find_all('td')[1:]
            nlst = [] 
            for cell in row:
                nlst.append(cell.get_text().strip())
            lst.append(nlst)
            
        str_lst = str(lst).replace('[', '{').replace(']', '}')
        
        # Convert the dictionary to a string
        table_str = str(str_lst)
   
        # Convert the tag object to a string
        tag_str = str(table)
   
        # Concatenate the table string with the remaining HTML content
        output_html = str(soup).replace(tag_str, table_str)
        # output_html = html_content.replace('</body>', f'{table_str} .\n </body>')
        
    else: 
        # Extract data from the table
        rows = table.find_all('tr')
        header_row = rows[0]
        headers = [header.text.strip() for header in header_row.find_all('td')[1:]]  # Exclude the first column
        
        results = [] 
        for row in rows[1:]:
            columns = row.find_all('td')
            category = columns[1].text.strip()
            result = []
            for header, column in zip(headers[1:], columns[2:]):
                result.append([header, column.text.strip()])
            results.append([category, result])
        
        str_results = str(results).replace('[', '{').replace(']', '}')
        
        # Convert the dictionary to a string
        table_str = str(str_results)
   
        # Convert the tag object to a string
        tag_str = str(table)
   
        # Concatenate the table string with the remaining HTML content
        output_html = str(soup).replace(tag_str, table_str)
        # output_html = html_content.replace('</body>', f'{table_str} .\n </body>')
    return output_html 

In [20]:
merged_df.shape

(17612, 5)

In [21]:
for i, row in tqdm(merged_df.iterrows()):
    output = convert_table_to_pseudo_json(row)
    merged_df.at[i, 'pseudo_json_encoding'] = output

17612it [00:56, 312.16it/s]


In [22]:
merged_df.shape

(17612, 6)

In [23]:
merged_df.head()

,answer,html_content,index,json_encoding,question,pseudo_json_encoding
0,17739,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under stock and option pl...,"<html>\n<body>\n ""distribution date"" ) . unti..."
1,18851,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is total ?,"<html>\n<body>\n ""distribution date"" ) . unti..."
2,842,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under the vertex purchase...,"<html>\n<body>\n ""distribution date"" ) . unti..."
3,270,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under the vertex 401 ( k ...,"<html>\n<body>\n ""distribution date"" ) . unti..."
4,$0.88,"<html>\n <body>\n ""three factor formula"" ) . ...",2,"<html>\n<body>\n ""three factor formula"" ) . t...",What is basic earnings ( loss ) per share for ...,"<html>\n<body>\n ""three factor formula"" ) . t..."


In [24]:
# merged_df = merged_df.drop('html_content', axis='columns')

In [25]:
merged_df.head()

,answer,html_content,index,json_encoding,question,pseudo_json_encoding
0,17739,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under stock and option pl...,"<html>\n<body>\n ""distribution date"" ) . unti..."
1,18851,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is total ?,"<html>\n<body>\n ""distribution date"" ) . unti..."
2,842,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under the vertex purchase...,"<html>\n<body>\n ""distribution date"" ) . unti..."
3,270,"<html>\n <body>\n ""distribution date"" ) . unt...",1,"<html>\n<body>\n ""distribution date"" ) . unti...",What is common stock under the vertex 401 ( k ...,"<html>\n<body>\n ""distribution date"" ) . unti..."
4,$0.88,"<html>\n <body>\n ""three factor formula"" ) . ...",2,"<html>\n<body>\n ""three factor formula"" ) . t...",What is basic earnings ( loss ) per share for ...,"<html>\n<body>\n ""three factor formula"" ) . t..."


In [26]:
for index, row in merged_df.iterrows():
    print(index)
    print(row['pseudo_json_encoding'])
    print(row['question'])
    print(row['answer'])
    print('***')
    if index == 20:
        break

0
<html>
<body>
  "distribution date" ) . until the distribution date ( or earlier redemption or expiration of the rights ) , the rights will be traded with , and only with , the common stock . until a right is exercised , the right will not entitle the holder thereof to any rights as a stockholder . if any person or group becomes an acquiring person , each holder of a right , other than rights beneficially owned by the acquiring person , will thereafter have the right to receive upon exercise and payment of the purchase price that number of shares of common stock having a market value of two times the purchase price and , if the company is acquired in a business combination transaction or 50% ( 50 % ) or more of its assets are sold , each holder of a right will thereafter have the right to receive upon exercise and payment of the purchase price that number of shares of common stock of the acquiring company which at the time of the transaction will have a market value of two times the 

In [27]:
merged_df = merged_df.drop('json_encoding', axis='columns')
merged_df.head()

,answer,html_content,index,question,pseudo_json_encoding
0,17739,"<html>\n <body>\n ""distribution date"" ) . unt...",1,What is common stock under stock and option pl...,"<html>\n<body>\n ""distribution date"" ) . unti..."
1,18851,"<html>\n <body>\n ""distribution date"" ) . unt...",1,What is total ?,"<html>\n<body>\n ""distribution date"" ) . unti..."
2,842,"<html>\n <body>\n ""distribution date"" ) . unt...",1,What is common stock under the vertex purchase...,"<html>\n<body>\n ""distribution date"" ) . unti..."
3,270,"<html>\n <body>\n ""distribution date"" ) . unt...",1,What is common stock under the vertex 401 ( k ...,"<html>\n<body>\n ""distribution date"" ) . unti..."
4,$0.88,"<html>\n <body>\n ""three factor formula"" ) . ...",2,What is basic earnings ( loss ) per share for ...,"<html>\n<body>\n ""three factor formula"" ) . t..."


In [28]:
merged_df.to_csv('../data/pseudo_json_dataset_V1.csv')